In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Get the API key and model from .env
groq_api_key = os.getenv("GROQ_API_KEY")
groq_model = os.getenv("GROQ_MODEL")

# Set environment variables for use in code
os.environ["GROQ_API_KEY"] = groq_api_key
os.environ["GROQ_MODEL"] = groq_model

print("Using Groq model:", groq_model)


Using Groq model: openai/gpt-oss-120b


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import LLM after setting environment variables
from crewai.llm import LLM

# Create the LLM instance with reduced max_tokens to avoid 8000 limit
llm = LLM(
    model=f"groq/{groq_model}",
    api_key=groq_api_key,
    temperature=0.7,
    timeout=60,
    max_retries=3
)

In [4]:
from crewai import Agent, Task, Crew

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## writer Agent

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

## Editor Agent

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

## Plan Task

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.keep under 250 words.",
    agent=planner,
)

## Writer Task

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.keep under 600 words.",
    agent=writer,
)

## Editor Task

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.keep under 600 words.",
    agent=editor
)

## Create the Crew

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True,
    memory=False,  # Disable memory to save tokens
    cache=True     # Enable cache to avoid repeated API calls
)


## Running the crew

In [12]:
import time
# Add delay before execution to avoid rate limits
time.sleep(2)

# Execute with try-catch for better error handling
try:
    result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})
    print(result)
except Exception as e:
    if "token" in str(e).lower() or "rate" in str(e).lower():
        print("Rate limit or token limit reached. Waiting 30 seconds and retrying...")
        time.sleep(30)
        try:
            result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})
            print(result)
        except Exception as retry_error:
            print(f"Retry failed: {retry_error}")
    else:
        print(f"Error occurred: {e}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b4098160-889d-4d4a-97a0-aa2dfc386839                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **                                                                                                             │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## 1. Target Audience & Pain Points                                                                            │
│  | Segment | Interests | Core Pain Points |                                                                     │
│  |---------|-----------|------------------|                                                                     │
│  | **Business Executives** | ROI of AI, competitive advantage | High upfront cost, talent shortage, regulatory  │
│  risk |                                                                                                         │
│  | **Tech Professionals & Data Scientists** | Cutting‑edge models, tooling | Keeping skills current,            │
│  integration complexity |                                                                                       │
│  | **Marketers & Product Managers** | AI‑driven content, personalization | Measuring impact, ethical misuse |   │
│  | **Students & Early‑career** | Learning pathways, certifications | Over‑information, unclear career roadmap   │
│  |                                                                                                              │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## 2. SEO Keywords (high‑search, low‑competition)                                                              │
│  - artificial intelligence trends 2025                                                                          │
│  - generative AI use cases                                                                                      │
│  - AI regulation EU AI Act 2024                                                                                 │
│  - foundation models OpenAI GPT‑4.5                                                                             │
│  - AI adoption roadmap for enterprises                                                                          │
│  - AI ethics and bias mitigation                                                                                │
│  - edge AI hardware Nvidia                                                                                      │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## 3. Content Outline (≈ 1,500‑2,000 words)                                                                    │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: fdf0dc6d-2ac4-415d-a326-3774ffb597bb                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.                      │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Artificial Intelligence in 2025: Trends, Risks, and a Roadmap for Leaders                                    │
│                                                                                                                 │
│  *By *Your Brand* – Thought leadership for executives, technologists, and innovators*                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## Introduction: Why AI Matters Now                                                                            │
│                                                                                                                 │
│  Artificial intelligence is no longer a futuristic buzzword; it is a strategic imperative. **73 % of CEOs**     │
│  plan to launch a major AI initiative by the end of 2025, attracted by higher margins and faster product        │
│  cycles. This post delivers a concise look at **artificial intelligence trends 2025**, practical **generative   │
│  AI use cases**, and a clear **AI adoption roadmap for enterprises**.                                           │
│                                                                                                                 │
│  > **Note:** Waiting for a “perfect” AI solution can cost you the competitive edge early adopters are already   │
│  harvesting.                                                                                                    │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## Latest Trends Shaping the Landscape                                                                         │
│                                                                                                                 │
│  ### Generative AI & Foundation Models                                                                          │
│  The debut of **OpenAI GPT‑4.5** and **Claude 3** has lowered the barrier to high‑quality content, code, and    │
│  data synthesis. Retailers use these models for personalized product descriptions, while marketers automate     │
│  multi‑channel copy, cutting time‑to‑publish by roughly 30 %.                                                   │
│                                                                                                                 │
│  ### Edge AI & TinyML                                                                                           │
│  Advances in **edge AI hardware from Nvidia**, especially the Jetson series, now enable on‑device inference,    │
│  reducing latency and privacy risks. Manufacturers are deploying TinyML sensors to predict equipment failures   │
│  in real time, a capability that previously required costly cloud pipelines.                                    │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: fc5294f7-696e-4d05-b73e-29ba3e17c707                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b4098160-889d-4d4a-97a0-aa2dfc386839                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Artificial Intelligence in 2025: Trends, Risks, and a Roadmap for Leaders                      │
│                                                                                                                 │
│  *By *Your Brand* – Thought leadership for executives, technologists, and innovators*                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## Introduction: Why AI Matters Now                                                                            │
│                                                                                                                 │
│  Artificial intelligence is no longer a futuristic buzzword; it is a strategic imperative. **73 % of CEOs**     │
│  plan to launch a major AI initiative by the end of 2025, attracted by higher margins and faster product        │
│  cycles. This post delivers a concise look at **artificial intelligence trends 2025**, practical **generative   │
│  AI use cases**, and a clear **AI adoption roadmap for enterprises**.                                           │
│                                                                                                                 │
│  > **Note:** Waiting for a “perfect” AI solution can cost you the competitive edge early adopters are already   │
│  harvesting.                                                                                                    │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## Latest Trends Shaping the Landscape                                                                         │
│                                                                                                                 │
│  ### Generative AI & Foundation Models                                                                          │
│  The debut of **OpenAI GPT‑4.5** and **Claude 3** has lowered the barrier to high‑quality content, code, and    │
│  data synthesis. Retailers use these models for personalized product descriptions, while marketers automate     │
│  multi‑channel copy, cutting time‑to‑publish by roughly 30 %.                                                   │
│                                                                                                                 │
│  ### Edge AI & TinyML                                                                                           │
│  Advances in **edge AI hardware from Nvidia**, especially the Jetson series, now enable on‑device inference,    │
│  reducing latency and privacy risks. Manufacturers are deploying TinyML sensors to predict equipment failures   │
│  in real time, a capability that previously required c

# Artificial Intelligence in 2025: Trends, Risks, and a Roadmap for Leaders  

*By *Your Brand* – Thought leadership for executives, technologists, and innovators*  

---  

## Introduction: Why AI Matters Now  

Artificial intelligence is no longer a futuristic buzzword; it is a strategic imperative. **73 % of CEOs** plan to launch a major AI initiative by the end of 2025, attracted by higher margins and faster product cycles. This post delivers a concise look at **artificial intelligence trends 2025**, practical **generative AI use cases**, and a clear **AI adoption roadmap for enterprises**.  

> **Note:** Waiting for a “perfect” AI solution can cost you the competitive edge early adopters are already harvesting.  

---  

## Latest Trends Shaping the Landscape  

### Generative AI & Foundation Models  
The debut of **OpenAI GPT‑4.5** and **Claude 3** has lowered the barrier to high‑quality content, code, and data synthesis. Retailers use these models for personalized product descri

In [13]:
from IPython.display import Markdown

# Remove the markdown code fences so it renders correctly
clean_text = result.raw.replace("```markdown", "").replace("```", "")

Markdown(clean_text)


# Artificial Intelligence in 2025: Trends, Risks, and a Roadmap for Leaders  

*By *Your Brand* – Thought leadership for executives, technologists, and innovators*  

---  

## Introduction: Why AI Matters Now  

Artificial intelligence is no longer a futuristic buzzword; it is a strategic imperative. **73 % of CEOs** plan to launch a major AI initiative by the end of 2025, attracted by higher margins and faster product cycles. This post delivers a concise look at **artificial intelligence trends 2025**, practical **generative AI use cases**, and a clear **AI adoption roadmap for enterprises**.  

> **Note:** Waiting for a “perfect” AI solution can cost you the competitive edge early adopters are already harvesting.  

---  

## Latest Trends Shaping the Landscape  

### Generative AI & Foundation Models  
The debut of **OpenAI GPT‑4.5** and **Claude 3** has lowered the barrier to high‑quality content, code, and data synthesis. Retailers use these models for personalized product descriptions, while marketers automate multi‑channel copy, cutting time‑to‑publish by roughly 30 %.  

### Edge AI & TinyML  
Advances in **edge AI hardware from Nvidia**, especially the Jetson series, now enable on‑device inference, reducing latency and privacy risks. Manufacturers are deploying TinyML sensors to predict equipment failures in real time, a capability that previously required costly cloud pipelines.  

### AI Regulation  
The **EU AI Act 2024** entered its implementation phase, mandating risk assessments for high‑impact systems. In the United States, the draft AI Bill of Rights is prompting firms to embed transparency and fairness checks early in the development cycle.  

> **Note:** Treating compliance as a differentiator can turn regulatory work into a market advantage.  

---  

## Key Players & Noteworthy News  

- **OpenAI** released GPT‑4.5 with a tiered API pricing model that rewards high‑volume, low‑latency workloads, making large‑scale deployment more affordable.  
- **Google DeepMind** launched Gemini 2, a multimodal model that excels at image‑text reasoning, opening new possibilities for AI‑driven design tools.  
- **Microsoft** integrated Azure AI Copilot across Office 365, showing how generative AI can augment everyday productivity without a steep learning curve.  
- **Anthropic** introduced Claude 3 for enterprise, emphasizing safety‑by‑design and built‑in bias mitigation features.  
- **Nvidia** rolled out the H100 Tensor Core GPU and an updated AI Enterprise Suite, reinforcing its role as the backbone for both cloud and edge AI workloads.  

---  

## Impact on Business: ROI, Skills, and Ethics  

A **McKinsey Global Institute** study finds that firms embedding AI in core processes achieve a **3× revenue lift** within two years, especially in retail and finance. Yet the **skills gap** persists: 58 % of tech leaders struggle to find qualified data scientists. Upskilling pathways—certifications on GPT‑4.5 and Nvidia’s edge platforms—are now essential components of any **AI adoption roadmap for enterprises**.  

Ethical considerations are equally critical. Applying an **AI ethics and bias mitigation** checklist can reduce model‑bias incidents by up to 40 %, according to a Harvard Business Review analysis. Robust data governance and regular audits are non‑negotiable under the **EU AI Act 2024**.  

---  

## Implementation Checklist: From Pilot to Scale  

1. **Define a high‑impact use case** (e.g., generative AI for customer support).  
2. **Run a short‑term pilot** with clear KPIs—cost reduction, response time, or conversion lift.  
3. **Measure outcomes** against baseline and refine the model or data pipeline.  
4. **Scale** using cloud or edge infrastructure (consider Nvidia hardware for latency‑sensitive apps).  
5. **Monitor compliance** continuously—track bias metrics, data provenance, and regulatory updates.  

---  

## Future Outlook: 2026‑2028 and Beyond  

From 2026 onward, AI‑driven sustainability solutions are expected to dominate, with models optimizing energy use in data centers and supply chains. Emerging markets in Africa and Southeast Asia will lean on **edge AI** to bypass limited connectivity, creating fresh growth frontiers. By 2028, the convergence of generative AI, foundation models, and responsible governance will reshape not only products but entire business models.  

> **Note:** Organizations that embed ethical AI practices now will avoid costly retrofits when expanding into these new markets.  

---  

## Call to Action  

Ready to accelerate your AI journey? **Download our free “AI Adoption Playbook,”** subscribe for weekly trend alerts, and register for our upcoming webinar **“Navigating the AI Regulation Landscape.”**  

---  

*Keywords used naturally: artificial intelligence trends 2025, generative AI use cases, AI regulation EU AI Act 2024, foundation models OpenAI GPT‑4.5, AI adoption roadmap for enterprises, AI ethics and bias mitigation, edge AI hardware Nvidia.*